In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from datetime import timedelta
import math
%matplotlib inline

In [3]:
filter_size1 = 5
num_filter1 = 16

filter_size2 = 5
num_filter2 = 36

fc_size = 128

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
data = input_data.read_data_sets('data/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
print("Training Set:\t {}".format(len(data.train.labels)))
print("Testing Set:\t {}".format(len(data.test.labels)))
print("Validation Set:\t {}".format(len(data.validation.labels)))

Training Set:	 55000
Testing Set:	 10000
Validation Set:	 5000


In [7]:
img_size = 28

img_size_flat = img_size*img_size

img_shape = (img_size,img_size)

num_channels = 1

num_classes = 10

In [71]:
data.test.cls = np.argmax(data.test.labels, axis=1)

In [8]:
#CNN with Tensorflow

In [9]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.05))

In [10]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05,shape=[length]))

In [11]:
#CNN

def new_conv_layer(input,
                  num_input_channels,
                  filter_size,
                  num_filters,
                  use_pooling=True):
    
    shape = [filter_size,filter_size, num_input_channels,num_filters]
    
    weights = new_weights(shape)
    
    biases = new_biases(length=num_filters)
    
    layer = tf.nn.conv2d(input=input,
                        filter = weights,
                        strides = [1,1,1,1],
                        padding=  'SAME')
    
    layer+=biases
    
    if use_pooling:
        layer = tf.nn.max_pool(value=layer,
                              ksize = [1,2,2,1],
                              strides = [1,2,2,1],
                              padding = 'SAME')
    
    layer = tf.nn.relu(layer)
    
    return layer,weights

In [12]:
def flatten_layer(layer):
    
    layer_shape = layer.get_shape()
    
    num_features = layer_shape[1:4].num_elements()
    
    layer_flat = tf.reshape(layer,[-1,num_features])
    
    return layer_flat,num_features

In [13]:
def new_fc_layer(input,
                num_inputs,
                num_output,
                use_relu=True):
    
    weights = new_weights(shape=[num_inputs,num_output])
    biases = new_biases(length=num_output)
    
    layer = tf.matmul(input,weights) + biases
    
    if use_relu:
        layer = tf.nn.relu(layer)
        
    return layer

In [14]:
x = tf.placeholder(tf.float32,shape=[None,img_size_flat],name = 'x')

In [15]:
x

<tf.Tensor 'x:0' shape=(?, 784) dtype=float32>

In [16]:
x_image = tf.reshape(x,[-1,img_size,img_size,num_channels])

In [17]:
y_true = tf.placeholder(tf.float32,shape=[None,10],name = 'y_true')

In [18]:
y_true_cls = tf.argmax(y_true,dimension=1)

Instructions for updating:
Use the `axis` argument instead


In [19]:
layer_conv1,weights_conv1 = new_conv_layer(input=x_image,
                                           num_input_channels=num_channels,
                                           filter_size=filter_size1,
                                           num_filters = num_filter1,
                                          use_pooling=True)

In [20]:
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 14, 14, 16) dtype=float32>

In [21]:
layer_conv2, weights_conv2 = new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filter1,
                   filter_size=filter_size2,
                   num_filters=num_filter2,
                   use_pooling=True)

In [22]:
layer_conv2

<tf.Tensor 'Relu_1:0' shape=(?, 7, 7, 36) dtype=float32>

In [23]:
layer_flat, num_features = flatten_layer(layer_conv2)

In [24]:
layer_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 1764) dtype=float32>

In [25]:
num_features

1764

In [26]:
layer_fc1= new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_output=fc_size,
                         use_relu=True)

In [27]:
layer_fc1

<tf.Tensor 'Relu_2:0' shape=(?, 128) dtype=float32>

In [28]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                        num_inputs = fc_size,
                        num_output = num_classes,
                        use_relu=True)

In [29]:
layer_fc2

<tf.Tensor 'Relu_3:0' shape=(?, 10) dtype=float32>

In [30]:
y_pred = tf.nn.softmax(layer_fc2)

In [31]:
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [32]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,labels=y_true)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [33]:
cost =  tf.reduce_mean(cross_entropy)

In [34]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [35]:
correct_prediction = tf.equal(y_pred_cls,y_true_cls)

In [36]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [37]:
#Session

In [38]:
sess = tf.Session()

In [39]:
sess.run(tf.global_variables_initializer())

In [40]:
training_batch_size = 64

In [74]:
total_iterations = 0

def optimize(num_iteration):
    
    global total_iterations
    
    start_time = time.time()
    
    for i in range(total_iterations,total_iterations +  num_iteration):
        
        x_batch,y_true_batch = data.train.next_batch(training_batch_size)
        
        feed_dict_train = {x:x_batch, y_true:y_true_batch}
        
        sess.run(optimizer,feed_dict=feed_dict_train)
        
        if i%100==0:
            
            acc = sess.run(accuracy,feed_dict=feed_dict_train)
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"
            
            print(msg.format(i+1,acc))
            
            end_time = time.time()
        
            time_diff = end_time - start_time
        
            print("Time usage: " + str(timedelta(seconds=int(round(time_diff)))))
        
        total_iterations+=num_iteration
        
       

In [72]:
test_batch_size = 256

def print_test_accuracy():
    
    num_test = len(data.test.images)
    
    cls_pred = np.zeros(shape=num_test,dtype=np.int)
    
    i = 0
    
    while i<num_test:
        
        j = min(i+test_batch_size,num_test)
        
        images = data.test.images[i:j,:]
        
        labels = data.test.labels[i:j,:]
        
        feed_dict = {x:images,y_true:labels}
        
        cls_pred[i:j] = sess.run(y_pred_cls,feed_dict=feed_dict)
        
        i=j
    
    cls_true = data.test.cls
    
    correct = (cls_pred==cls_true)
    
    correct_sum = np.sum(correct)
    
    acc = float(correct_sum)/num_test
    
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

In [75]:
optimize(num_iteration=999)

Optimization Iteration:      1, Training Accuracy:  60.9%
Time usage: 0:00:00
Optimization Iteration:    101, Training Accuracy:  64.1%
Time usage: 0:00:15
Optimization Iteration:    201, Training Accuracy:  82.8%
Time usage: 0:00:32
Optimization Iteration:    301, Training Accuracy:  79.7%
Time usage: 0:00:51
Optimization Iteration:    401, Training Accuracy:  73.4%
Time usage: 0:01:11
Optimization Iteration:    501, Training Accuracy:  71.9%
Time usage: 0:01:30
Optimization Iteration:    601, Training Accuracy:  70.3%
Time usage: 0:01:49
Optimization Iteration:    701, Training Accuracy:  65.6%
Time usage: 0:02:13
Optimization Iteration:    801, Training Accuracy:  70.3%
Time usage: 0:02:34
Optimization Iteration:    901, Training Accuracy:  68.8%
Time usage: 0:02:56


In [76]:
print_test_accuracy()

Accuracy on Test-Set: 74.1% (7414 / 10000)
